In [1]:
import numpy as np
import pandas as pd
import pickle
# SQL related packages
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
# sklearn packages
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from sklearn.linear_model import LogisticRegression as Log
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict, GridSearchCV
# text analysis packages
from nltk import word_tokenize
from nltk.corpus import stopwords
import string
from collections import Counter
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# access to sql database
dbname = 'patent_db'
username = 'jy'
pswd = 'jy'

engine = create_engine('postgresql://%s:%s@localhost/%s'%(username,pswd,dbname))

# reading from sql database
# connect:
con = None
con = psycopg2.connect(database = dbname, user = username, host='localhost', password=pswd)

#### Extract claims

In [3]:
# read data from 2004-2007
years = np.arange(2004, 2008)

# dataframe to store the results
claims = pd.DataFrame()

# import the abstract from each table
for year in years:
    # query:
    sql_query = """
    SELECT claims, id, payment_times
        FROM patents_%s;
    """ %str(year)

    results = pd.read_sql_query(sql_query,con)
    
    claims = pd.concat([claims, results], axis = 0)
    
# check size of the data
claims.shape

(12033, 3)

In [6]:
# extract the response variable
# reformat the response variable into binary
y_data = np.zeros(claims.shape[0])
y_data[claims['payment_times'].values >= 2] = 1

print "Percentage of patents with > 1 maintenance fee payments: ", np.mean(y_data)

Percentage of patents with > 1 maintenance fee payments:  0.628521565694


#### Tokenization and cleaning

In [7]:
def tokenize_cleaning(text):
    # tokenize the text first
    tokens = word_tokenize(text.decode('utf-8'))
    
    # lowercase all the words
    tokens = [w.lower() for w in tokens]
    
    # clean up stop words and punctuations 
    stop_list = stopwords.words('english') + list(string.punctuation)

    tokens_no_stop = [token for token in tokens
                        if token not in stop_list]            
    
#     # extract stem of the words
#     stemmer = PorterStemmer()
#     tokens_stem = [stemmer.stem(token) for token in tokens_no_stop]

    # use lemma instead
    # reason: remove the influence of plural or tense
    # but retain the subtle difference in legal writting
    lemmatizer = WordNetLemmatizer()
    tokens_lemma = [lemmatizer.lemmatize(token) for token in tokens_no_stop]
    
    # remove numbers (the actual values are not useful)
    tokens_no_num = []
    for token in tokens_lemma:
        try:
            float(token)
        except:
            tokens_no_num.append(token)
    
    return tokens_no_num

In [8]:
# tokenize_clean the abstracts and count the occurence of the words

cleaned_text = []
for i in range(claims.shape[0]):
    tokens = tokenize_cleaning(claims['claims'].iloc[i])
    cleaned_text.append(' '.join(word for word in tokens))

In [18]:
# convert to bag-of-words
# min number selected by examining the low-frequency words
vectorizer = CountVectorizer(max_df = 0.4, min_df=5)

# perform a count-based vectorization of the document
word_vec = vectorizer.fit(cleaned_text)
word_counts = word_vec.transform(cleaned_text)

# convert to array
word_counts = word_counts.toarray()
word_counts.shape

(12033, 8035)

In [19]:
# still need to remove some number words 
# due to how CountVectorizer treats '-' and '/'

# remove any word with numbers in it
words = word_vec.get_feature_names()
num_word_index = np.zeros(len(words))

for i in range(len(words)):
    word = words[i]
    for j in range(len(word)):
        try:
            float(word[j])
            num_word_index[i] = 1
            break
        except:
            continue
        
print "Number of numerical words detected: ", int(np.sum(num_word_index))

# remove the number words
words_no_num = np.asarray(words)[num_word_index == 0]
word_counts = word_counts[:, num_word_index == 0]

word_counts.shape

Number of numerical words detected:  222


(12033, 7813)

In [20]:
# check the total counts for each word
total_counts = np.sum(word_counts, axis = 0)

df = pd.DataFrame({
    'word': words_no_num,
    'count': total_counts
})

In [21]:
df.sort('count')

/home/jy/anaconda2/envs/my_projects_env/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,count,word
1992,5,develops
1668,5,crapemyrtle
3751,5,insulate
1637,5,corticotropin
716,5,bb
3986,5,know
4058,5,lease
4083,5,liberated
6338,5,sedimentation
5132,5,pentane


In [115]:
# tf_vectorizer = TfidfVectorizer()
# tfidf_article = tf_vectorizer.fit_transform(cleaned_text)
# tfidf_article = tfidf_article.toarray()

# print tfidf_article.shape

(12033, 29829)


#### Spliting train-test data and subsampling

In [23]:
# split train and test sets
x_train, x_test, y_train, y_test = train_test_split(word_counts, y_data, 
                                                    test_size = 0.2, 
                                                    random_state = 123)

print "Dataset dimensions:"
print "x_train: ", x_train.shape
print "x_test: ", x_test.shape
print "y_train: ", y_train.shape
print "y_test: ", y_test.shape

Dataset dimensions:
x_train:  (9626, 7813)
x_test:  (2407, 7813)
y_train:  (9626,)
y_test:  (2407,)


In [25]:
### subsampling the training data
# sample the same number of'useful' patents as the 'not useful' patents
# size of each class
num_size = np.sum(y_train == 0)

#random shuffle the rows
n = x_train.shape[0]
perm = range(n)
np.random.shuffle(perm)

x_train = x_train[perm]
y_train = y_train[perm]

# separate the two classes
x_useful = x_train[y_train == 1, :]
x_not_useful = x_train[y_train == 0, :]
y_useful = y_train[y_train == 1]
y_not_useful = y_train[y_train == 0]

# sample num_size from the 'useful' class
x_useful = x_useful[:num_size, :]
y_useful = y_useful[:num_size]

# combine the two classes
x_train_sub = np.concatenate((x_useful, x_not_useful), axis = 0)
y_train_sub = np.concatenate((y_useful, y_not_useful), axis = 0)

# shuffle again
# shuffle the combined data
n2 = x_train_sub.shape[0]
perm2 = range(n2)
np.random.shuffle(perm2)

x_train_sub = x_train_sub[perm2]
y_train_sub = y_train_sub[perm2]

# check the size
print x_train_sub.shape
print y_train_sub.shape

(7136, 7813)
(7136,)


In [26]:
# try Naive Bayes with Gaussian Distribution
gnb = GaussianNB()

# fit on the training data
gnb.fit(x_train_sub, y_train_sub)

GaussianNB(priors=None)

In [27]:
# predict on the test data
y_pred = gnb.predict(x_test)

In [28]:
# accuracy
np.mean(y_pred == y_test)

0.52762775238886583

In [29]:
metrics.confusion_matrix(y_test, y_pred)

array([[632, 270],
       [867, 638]])

### PCA

In [30]:
# try dimensionality reduction using PCA
pca = PCA()

x_train_pca = pca.fit_transform(x_train_sub)
x_test_pca = pca.transform(x_test)

In [31]:
# find the cum-variance explained at each level
total_var = np.cumsum(pca.explained_variance_ratio_)
n_pc = np.where((total_var > 0.9) == True)[0][0]

In [32]:
n_pc

434

In [40]:
### use logistic regression

# call the model function
model = Log()
# parameter tuning
c =  np.logspace(-5, 5, 11)

# use grid search with 5-fold CV
grid_model = GridSearchCV(model, param_grid = {'C': c}, cv  = 5, scoring = 'accuracy')
# fit on the data
grid_model = grid_model.fit(x_train_sub, y_train_sub) 
#grid_model = grid_model.fit(x_train_sub[:, :n_pc], y_train_sub) 

In [41]:
# check results
print "Best accuracy:", grid_model.best_score_
print "Best parameter: ", grid_model.best_params_

Best accuracy: 0.561939461883
Best parameter:  {'C': 0.0001}


In [42]:
# check confusion matrix
best_log = grid_model.best_estimator_
best_log.fit(x_train_sub[:, :n_pc], y_train_sub)
y_pred = best_log.predict(x_test[:, :n_pc])

# accuracy
print "Test accuracy: ", np.mean(y_pred == y_test)
print "F1 score: ", metrics.f1_score(y_test, y_pred)
print "Precision: ", metrics.precision_score(y_test, y_pred)
print "Recall: ", metrics.recall_score(y_test, y_pred)
metrics.confusion_matrix(y_test, y_pred)

Test accuracy:  0.521811383465
F1 score:  0.567130500188
Precision:  0.653379549393
Recall:  0.500996677741


array([[502, 400],
       [751, 754]])

In [43]:
### tune random forest

model = RandomForestClassifier(n_estimators = 100)

# tune max_features
param_space = np.arange(2, 40, 5)

grid_model = GridSearchCV(model, n_jobs = 4, 
                          param_grid = {'max_features': param_space}, 
                          cv  = 5, scoring = 'accuracy')
# fit on the data
#grid_model = grid_model.fit(x_train_pca[:, :n_pc], y_train_sub)
grid_model = grid_model.fit(x_train_sub, y_train_sub)

In [44]:
# check results
print "Best accuracy:", grid_model.best_score_
print "Best parameter: ", grid_model.best_params_

Best accuracy: 0.56908632287
Best parameter:  {'max_features': 7}


In [46]:
# check confusion matrix
best_rf = grid_model.best_estimator_
best_rf.fit(x_train_sub, y_train_sub)
y_pred = best_rf.predict(x_test)

# accuracy
print "Test accuracy: ", np.mean(y_pred == y_test)
print "F1 score: ", metrics.f1_score(y_test, y_pred)
print "Precision: ", metrics.precision_score(y_test, y_pred)
print "Recall: ", metrics.recall_score(y_test, y_pred)
metrics.confusion_matrix(y_test, y_pred)

Test accuracy:  0.581221437474
F1 score:  0.630227439472
Precision:  0.703521703522
Recall:  0.570764119601


array([[540, 362],
       [646, 859]])